# Part 2:Graph Attention Network (GAT's)

## Other than the model architecture,the implementation is similiar to GCN's

In [ ]:
import energyflow
dataset = energyflow.qg_jets.load(num_data=100000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow') 
X = dataset[:-1][0]
y = dataset[-1]
sum_array_1 = []
sum_array_2 = []
sum_array_3 = []
sum_array_4 = []
for i in range(X.shape[0]):
    temp_1 = 0
    temp_2 = 0
    temp_3 = 0
    temp_4 = 0
    for j in range(X.shape[1]):
        if(X[i][j][0]!=0):
            temp_1 += X[i][j][0]
        if(X[i][j][1]!=0):
            temp_2 += X[i][j][1]
        if(X[i][j][2]!=0):
            temp_3 += X[i][j][2]
        if(X[i][j][3]!=0):
            temp_4 += X[i][j][3]
    sum_array_1.append(temp_1)
    sum_array_2.append(temp_2)
    sum_array_3.append(temp_3)
    sum_array_4.append(temp_4)
mean_1 = sum(sum_array_1)/len(sum_array_1)
mean_2 = sum(sum_array_2)/len(sum_array_2)
mean_3 = sum(sum_array_3)/len(sum_array_3)
mean_4 = sum(sum_array_4)/len(sum_array_4)
bool_1 = [i<=mean_1 for i in sum_array_1]
bool_2 = [i<=mean_2 for i in sum_array_2]
bool_3 = [i<=mean_3 for i in sum_array_3]
bool_4 = [i<=mean_4 for i in sum_array_4]

In [ ]:
binary_1 = []
for i in bool_1:
    if(i==True):
        binary_1.append(1)
    else:
        binary_1.append(0)
binary_2 = []
for i in bool_2:
    if(i==True):
        binary_2.append(1)
    else:
        binary_2.append(0)
binary_3 = []
for i in bool_3:
    if(i==True):
        binary_3.append(1)
    else:
        binary_3.append(0)
binary_4 = []
for i in bool_4:
    if(i==True):
        binary_4.append(1)
    else:
        binary_4.append(0)

In [ ]:
binary = []
for i in range(len(binary_1)):
    binary.append(binary_1[i] + binary_2[i] + binary_3[i] + binary_4[i])
connections_0 = []
connections_1 = []
connections_2 = []
connections_3 = []
connections_4 = []
for i in range(len(binary)):
    if(binary[i]==0):
        connections_0.append(i)
    if(binary[i]==1):
        connections_1.append(i)
    if(binary[i]==2):
        connections_2.append(i)
    if(binary[i]==3):
        connections_3.append(i)
    if(binary[i]==4):
        connections_4.append(i)

In [ ]:
labels = []
labels.append('Quarks')
labels.append('Gluons')
nodes = []
N = X.shape[0]
F = X.shape[2]
print('X shape: ', X.shape)
edge_list=[]
for i in range(len(connections_1)-1):
    edge_list.append((connections_1[i],connections_1[i+1]))
for i in range(len(connections_2)-1):
    edge_list.append((connections_2[i],connections_2[i+1]))
for i in range(len(connections_3)-1):
    edge_list.append((connections_3[i],connections_3[i+1]))
for i in range(len(connections_4)-1):
    edge_list.append((connections_4[i],connections_4[i+1]))
print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (100000, 139, 4)

Number of nodes (N):  100000

Number of features (F) of each node:  4

Categories:  {'Gluons', 'Quarks'}

Number of classes:  2


In [ ]:
import networkx as nx
#build the graph
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)

#obtain the adjacency matrix (A)
A = nx.adjacency_matrix(G)
print('Graph info: ', nx.info(G))

C:\Users\user\AppData\Local\Temp\ipykernel_7788\631010150.py:8: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


Graph info:  Graph with 94576 nodes and 94572 edges


C:\Users\user\AppData\Local\Temp\ipykernel_7788\631010150.py:9: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('Graph info: ', nx.info(G))


In [ ]:
import numpy as np
def limit_data(labels,limit=5000,val_num=0,test_num=5000):
    '''
    Get the index of train, validation, and test data
    '''
    label_counter = dict((l, 0) for l in labels)
    train_idx = []

    for i in range(len(labels)):
        label = labels[i]
        if label_counter[label]<limit:
            #add the example to the training data
            train_idx.append(i)
            label_counter[label]+=1
        
        #exit the loop once we found 20 examples for each class
        if all(count == limit for count in label_counter.values()):
            break
    
    #get the indices that do not go to traning data
    rest_idx = [x for x in range(len(labels)) if x not in train_idx]
    val_idx = rest_idx[:val_num]
    test_idx = rest_idx[val_num:(val_num+test_num)]
    return train_idx, val_idx,test_idx

train_idx,val_idx,test_idx = limit_data(labels)

#set the mask
train_mask = np.zeros((N,),dtype=bool)
train_mask[train_idx] = True

val_mask = np.zeros((N,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((N,),dtype=bool)
test_mask[test_idx] = True

In [ ]:
!pip install torch_geometric

In [ ]:
!pip install torch

In [ ]:
!pip install torch_sparse

In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=2)
        self.conv2 = GATConv(hidden_channels * 2, out_channels, heads=4)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.elu(self.conv2(x, edge_index))
        return F.log_softmax(x, dim=1)

In [ ]:
# Parameters
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping

In [ ]:
import torch
from torch_geometric.data import Data

# Convert the renormalized adjacency matrix A to a sparse tensor format
row, col = A.nonzero()
edge_index = torch.tensor([row, col], dtype=torch.long)

# Convert X and y to PyTorch tensors
X = torch.tensor(X, dtype=torch.float)
X_train = X.reshape(100000, 139*4)
y = torch.tensor(y, dtype=torch.long)

# Create a PyTorch Geometric data object
data = Data(x=X_train, edge_index=edge_index,y=y)

# Set the train and test masks as attributes of the data object
data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
data.test_mask = torch.tensor(test_mask, dtype=torch.bool)

# Initialize the GCN model
model = GAT(in_channels=139*4, hidden_channels=128, out_channels = 2)

# Define the loss function and optimizer
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(25):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

C:\Users\user\AppData\Local\Temp\ipykernel_1460\3708665358.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float)
C:\Users\user\AppData\Local\Temp\ipykernel_1460\3708665358.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long)


Epoch 1, Loss: 884.0875
Epoch 2, Loss: 55.2798
Epoch 3, Loss: 601.2061
Epoch 4, Loss: 397.2829
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 251.6613
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 26.9728
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000
Epoch 11, Loss: 0.0000
Epoch 12, Loss: 0.0000
Epoch 13, Loss: 0.0000
Epoch 14, Loss: 0.0000
Epoch 15, Loss: 0.0000
Epoch 16, Loss: 0.0000
Epoch 17, Loss: 0.0000
Epoch 18, Loss: 0.0000
Epoch 19, Loss: 0.0000
Epoch 20, Loss: 0.0000
Epoch 21, Loss: 0.0000
Epoch 22, Loss: 0.0000
Epoch 23, Loss: 0.0000
Epoch 24, Loss: 29.4681
Epoch 25, Loss: 0.0000


# I have tried multiple model architectures with different layers.However,unlike GCNs,the loss in the GAT model was not converging to a single value as the model was not being trained properly. I tried to increase the complexity of the model architecture to capture the essential relationships between these nodes,but the model seems to be overfitting.If the number of layers is low, it would be insufficient for message passing due to the extremely high number of nodes. Fine tuning the model and the model architecture by adding more dropout layers to prevent overfitting were also tried.

# In conclusion, GAT models were harder to fine tune due to the high number of parameters to prevent overfitting compared to GCNs and GCN model was found to be relatively more efficient although the contrary was expected as GATs are thought to be more effective for bigger datasets compared to GCNs.